In [29]:
import os
import textwrap
import numpy as np
from PIL import Image, ImageDraw, ImageFont
import imageio
from moviepy.editor import VideoFileClip, AudioFileClip
from moviepy.editor import concatenate_audioclips



In [ ]:
# === CONFIG ===
comic_panels = [
    {"image": "1. morning-light.jpg", "text": "Another day, same desk… and yet something about today feels slightly off..."},
    {"image": "2. opening-light.jpg", "text": "Every dataset feels like a new landscape — but this one’s… different. Rows missing.. Patterns too neat.. hmmm weird!"},
    {"image": "3. frown.jpg", "text": "Okay, who cleans up data this perfectly? Definitely not me before coffee...."},
    {"image": "4. file.jpg", "text": "One moment I’m buried in dashboards, then out of nowhere, a file blinks on my desktop pops up."},
    {"image": "5. Leaning.jpg", "text": "Tempting!! Probably IT maintenance or could be… a dare??"},
    {"image": "6. Clicks_open.jpg", "text": "This is an empty sheet. Except two words: 'Look outside.'"},
    {"image": "7. Look-outside.jpg", "text": "Okay, nothing unusual. Just rain and my neighbor’s cat judging me again."},
    {"image": "8. Turning-back.jpg", "text": "Except… now the sheet says ‘Look closer.’"},
    {"image": "9. New-tab.jpg", "text": "There’s a new tab open. I swear it wasn’t there before."},
    {"image": "10. typing.jpg", "text": "The cursor moves - slow, deliberate- typing on its own, but my hands aren’t on the keyboard."},
    {"image": "11. glitch.jpg", "text": "Maybe it’s just a glitch… or maybe my thoughts finally learned to type back."},
    {"image": "12. glow.jpg", "text": "what exactly is this file? and how the cursor is moving on its own?"},
    {"image": "13. End.jpg", "text":  "And suddenly the last cell flashes: ‘You’re next.’"}
]

video_file = "comic_video.mp4"
final_video = "comic_with_music.mp4"
audio_file = "background.mp3"  # 🎵 your downloaded background music
font_path = "comic.ttf"
bubble_scale = 0.42
line_spacing = 16
fps = 10
display_seconds = 5
fade_frames = 10

In [31]:
# === DRAW SPEECH BUBBLE ===
def draw_comic_bubble(draw, x, y, w, h, outline_color=(0, 0, 0, 255), fill_color=(255, 255, 255, 200)):
    radius = h // 4
    draw.rounded_rectangle([x, y, x + w, y + h],
                           radius=radius, fill=fill_color,
                           outline=outline_color, width=2)

In [32]:
# === PANEL RENDERING ===
def draw_panel_with_bubble(panel):
    img = Image.open(panel["image"]).convert("RGBA")
    panel_w, panel_h = img.size
    draw = ImageDraw.Draw(img, 'RGBA')

    try:
        font = ImageFont.truetype(font_path, 32)
    except:
        font = ImageFont.truetype("arial.ttf", 32)

    # Bubble dimensions
    bubble_w = int(panel_w * bubble_scale)
    bubble_h = int(panel_h * bubble_scale * 0.5)
    bubble_y = 20
    bubble_x = int(panel_w * 0.15)

    # Draw bubble
    draw_comic_bubble(draw, bubble_x, bubble_y, bubble_w, bubble_h)

    # === Fit text dynamically ===
    text = panel["text"]
    max_font_size = 32
    min_font_size = 16
    best_font = font
    wrapped_text = text
    for size in range(max_font_size, min_font_size - 1, -2):
        current_font = ImageFont.truetype(font_path, size) if os.path.exists(font_path) else ImageFont.truetype("arial.ttf", size)
        chars_per_line = max(10, bubble_w // (size * 0.6))
        wrapped = textwrap.fill(text, width=int(chars_per_line))
        text_lines = wrapped.split("\n")
        line_height = current_font.getbbox("A")[3] - current_font.getbbox("A")[1]
        total_height = len(text_lines) * (line_height + line_spacing) - line_spacing

        if total_height <= bubble_h * 0.85:  # fits comfortably
            best_font = current_font
            wrapped_text = wrapped
            break

    # === Draw text ===
    text_lines = wrapped_text.split("\n")
    line_height = best_font.getbbox("A")[3] - best_font.getbbox("A")[1]
    text_block_height = (line_height + line_spacing) * len(text_lines) - line_spacing
    text_y = bubble_y + (bubble_h - text_block_height) // 2

    for line in text_lines:
        text_w = draw.textlength(line, font=best_font)
        text_x = bubble_x + (bubble_w - text_w) // 2
        draw.text((text_x, text_y), line, font=best_font, fill=(0, 0, 0, 255))
        text_y += line_height + line_spacing

    return img.convert("RGB")


In [33]:
# === CREATE VIDEO ===
def create_video_from_panels(panels, video_output, fps=10, display_seconds=4, fade_frames=10):
    frames = []
    images = [draw_panel_with_bubble(panel) for panel in panels]
    display_frames = display_seconds * fps

    for i, img in enumerate(images):
        frames.extend([np.array(img)] * display_frames)
        if i < len(images) - 1:
            next_img = np.array(images[i+1])
            for f in range(1, fade_frames + 1):
                alpha = f / (fade_frames + 1)
                blended = np.array(img)*(1-alpha) + next_img*alpha
                frames.append(blended.astype(np.uint8))

    imageio.mimwrite(video_output, frames, fps=fps)
    print(f"✅ Video created successfully: {video_output}")

In [34]:
# === MERGE VIDEO + BACKGROUND MUSIC ===
def merge_video_audio(video_path, audio_path, output_path):
    if not os.path.exists(audio_path):
        print("⚠️ No background.mp3 found in the current folder. Skipping music merge.")
        os.rename(video_path, output_path)
        return

    print("🎬 Adding background music...")
    video = VideoFileClip(video_path)
    audio = AudioFileClip(audio_path)

    # Loop or trim audio to match video duration
    if audio.duration < video.duration:
        # Loop the audio manually
        n_loops = int(np.ceil(video.duration / audio.duration))
        audio = concatenate_audioclips([audio] * n_loops)
    # Trim audio to exact video duration
    audio = audio.subclip(0, video.duration)

    # Combine video + audio
    final = video.set_audio(audio)
    final.write_videofile(output_path, codec="libx264", audio_codec="aac")
    print(f"🎵 Final video with background music saved as: {output_path}")


In [35]:
# === RUN ===
if __name__ == "__main__":
    create_video_from_panels(comic_panels, video_file,
                             fps=fps, display_seconds=display_seconds, fade_frames=fade_frames)
    merge_video_audio(video_file, audio_file, final_video)

✅ Video created successfully: comic_video.mp4
🎬 Adding background music...
Moviepy - Building video comic_with_music.mp4.
MoviePy - Writing audio in comic_with_musicTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video comic_with_music.mp4



Moviepy - Done !
Moviepy - video ready comic_with_music.mp4
🎵 Final video with background music saved as: comic_with_music.mp4
